<a href="https://colab.research.google.com/github/veryHapppy/study_ai/blob/main/Kaggle/backpack_prediction_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install optuna
!pip install catboost

In [33]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import RidgeCV
from sklearn.impute import SimpleImputer

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
import joblib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
'''
id => index
brand만으로는 차이없음




'''

'\nid => index\nbrand만으로는 차이없음\n\n\n\n\n'

In [35]:
def feature_engineering(data, train_df) :
  if 'Price' in data :
    data = data.drop('id', axis=1)
  else :
    data = data.set_index(data['id']).drop('id', axis=1)

  cat_cols = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
  num_cols = ['Weight Capacity (kg)', 'Compartments']

  cat_imputer = SimpleImputer(strategy='most_frequent')
  num_imputer = SimpleImputer(strategy='median')

  train_df[cat_cols] = cat_imputer.fit_transform(train_df[cat_cols])
  data[cat_cols] = cat_imputer.transform(data[cat_cols])
  train_df[num_cols] = num_imputer.fit_transform(train_df[num_cols])
  data[num_cols] = num_imputer.transform(data[num_cols])


  data['Material_Waterproof'] = data['Material'] + '_' + data["Waterproof"]
  data['Brand_Color'] = data['Brand'] + '_' + data['Color']
  data = pd.get_dummies(data, columns=['Material_Waterproof', 'Brand_Color', 'Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color'], drop_first=True)

  return data

In [36]:
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/[Kaggle 5-2] Backpack Prediction Challenge/train.csv")
extra_train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/[Kaggle 5-2] Backpack Prediction Challenge/training_extra.csv")

data_set = feature_engineering(extra_train_data, train_data)
train_sample = data_set.sample(frac=0.05, random_state=42)
tuning_X = train_sample.drop('Price', axis=1)
tuning_y = train_sample['Price']
X = data_set.drop('Price', axis=1)
y = data_set['Price']

In [37]:
def objective(trial) :
  xgb_params = {
    'n_estimators': trial.suggest_int('xgb_n', 250, 1000),
    'max_depth': trial.suggest_int('xgb_depth', 3, 7),
    'colsample_bytree': trial.suggest_float('xgb_colsample', 0.6, 0.8),
    'reg_alpha': trial.suggest_float('xgb_alpha', 1e-3, 10.0, log=True),
    'reg_lambda': trial.suggest_float('xgb_lambda', 1e-3, 10.0, log=True),
    'learning_rate': trial.suggest_float('xgb_lr', 0.005, 0.1),
  }
  lgbm_params = {
    'n_estimators': trial.suggest_int('lgbm_n', 250, 1000),
    'num_leaves': trial.suggest_int('lgbm_leaves', 15, 50),
    'learning_rate': trial.suggest_float('lgbm_lr', 0.005, 0.1),
    'verbose': -1
  }
  cat_params = {
    'iterations': trial.suggest_int('cat_iter', 250, 1000),
    'depth': trial.suggest_int('cat_depth', 3, 7),
    'learning_rate': trial.suggest_float('cat_lr', 0.005, 0.1),
    'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
    'random_strength': trial.suggest_float('random_strength', 0.1, 10.0),
    'logging_level': 'Silent' # CatBoost 조용히 시키기
  }
  w_xgb = trial.suggest_float('w_xgb', 0.5, 2.0)
  w_lgbm = trial.suggest_float('w_lgbm', 0.5, 2.0)
  w_cat = trial.suggest_float('w_cat', 0.5, 2.0)

  xgb = XGBRegressor(**xgb_params, tree_method='hist', device='cuda', n_jobs=-1, random_state=42)
  lgbm = LGBMRegressor(**lgbm_params, device='gpu', n_jobs=-1, random_state=42)
  cat = CatBoostRegressor(**cat_params, task_type='GPU', bootstrap_type='Poisson', random_state=42)

  voting_model = VotingRegressor(
    estimators=[('xgb', xgb), ('lgbm', lgbm), ('cat', cat)],
    weights=[w_xgb, w_lgbm, w_cat],
  )

  score = cross_val_score(voting_model, tuning_X, tuning_y, cv=3, scoring='neg_root_mean_squared_error')

  return -score.mean()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print(study.best_value)

[I 2026-02-15 06:07:05,576] A new study created in memory with name: no-name-531fc206-e590-4e4f-a27a-beb8bcace579
[I 2026-02-15 06:07:39,653] Trial 0 finished with value: 38.94686261999694 and parameters: {'xgb_n': 320, 'xgb_depth': 4, 'xgb_colsample': 0.7792610780165548, 'xgb_alpha': 0.05730066012208538, 'xgb_lambda': 0.001184501059446774, 'xgb_lr': 0.04501443029133618, 'lgbm_n': 493, 'lgbm_leaves': 50, 'lgbm_lr': 0.014050717636469534, 'cat_iter': 882, 'cat_depth': 5, 'cat_lr': 0.058172140199421324, 'l2_leaf_reg': 4.604968897599468, 'random_strength': 6.2816921536692965, 'w_xgb': 1.3671826802557439, 'w_lgbm': 0.8213590472590544, 'w_cat': 1.2732872203085073}. Best is trial 0 with value: 38.94686261999694.
[I 2026-02-15 06:08:09,465] Trial 1 finished with value: 38.97901506478572 and parameters: {'xgb_n': 386, 'xgb_depth': 3, 'xgb_colsample': 0.742065197553047, 'xgb_alpha': 0.006816562301871771, 'xgb_lambda': 0.25433717948992396, 'xgb_lr': 0.09085894713020741, 'lgbm_n': 525, 'lgbm_leave

38.933436763877474


In [40]:

best = study.best_params

xgb_final = XGBRegressor(
    n_estimators=best['xgb_n'],
    max_depth=best['xgb_depth'],
    learning_rate=best['xgb_lr'],
    colsample_bytree=best['xgb_colsample'],
    reg_alpha=best['xgb_alpha'],
    reg_lambda=best['xgb_lambda'],
    random_state=42,
    tree_method='hist',
    device='cuda',
)

lgbm_final = LGBMRegressor(
    n_estimators=best['lgbm_n'],
    num_leaves=best['lgbm_leaves'],
    learning_rate=best['lgbm_lr'],
    random_state=42,
    verbose=-1,
    device='gpu',
)

cat_final = CatBoostRegressor(
    iterations=best['cat_iter'],
    depth=best['cat_depth'],
    learning_rate=best['cat_lr'],
    l2_leaf_reg=best['l2_leaf_reg'],
    random_strength=best['random_strength'],
    task_type='GPU', # GPU 사용
    devices='0',
    random_state=42,
    logging_level='Silent'
)

voting_model = VotingRegressor(
    estimators=[('xgb', xgb_final), ('lgbm', lgbm_final), ('cat', cat_final)],
    weights=[best['w_xgb'], best['w_lgbm'], best['w_cat']]
)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

voting_model.fit(X_train, y_train)

voting_pred = voting_model.predict(X_test)

print(f'voting 오차 : {root_mean_squared_error(y_test, voting_pred)}')

joblib.dump(voting_model, 'voting_best.pkl')


voting 오차 : 38.90203126953337


['voting_best.pkl']

In [43]:

test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[Kaggle 5-2] Backpack Prediction Challenge/test.csv')
X = feature_engineering(test_data, train_data)

model = joblib.load('voting_best.pkl')
pred = model.predict(X)

submission = pd.DataFrame({
    'id': X.index,
    'Price': pred
})

submission.to_csv('submission.csv', index=False)
print("파일 생성 완료")

파일 생성 완료
